<a href="https://colab.research.google.com/github/fhac-ewi/recurrent-neural-network/blob/main/Uebung_7_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Übung 7 - Rekurrente neuronale Netze (RNNs)

## In dieser Übung ...
... beschäftigen wir uns mit rekurrenten neuronalen Netzen, die unter anderem für die Textvorhersage (`text prediction`) eingesetzt werden können. In dieser Übung werden Sie  ein eigenes Modell zur Vorhersage weiterer Buchstaben (Zeichen) zu einer gegebenen Buchstabensequenz erstellen. 

Dazu werden Sie einen Datensatz für das Training vorbereiten, ein Standard-RNN mithilfe von Keras implementieren und für die Vorhersage des nächsten Zeichens und sogar ganzen Sätzen nutzen.

Wenn Sie diese ersten Schritte geschafft haben und noch Zeit haben, dürfen Sie die beiden weiteren Varianten (LSTM und GRU) implementieren und anschließend die Genauigkeit (Accuracy) der Netze miteinander vergleichen. 



## 7.0 Vorbereitung

In diesem Abschnitt müssen Sie nichts programmieren! 🎉

Wir haben bereits alle notwendigen Imports für diese Übung hinzugefügt, sodass Sie direkt starten können. Sie müssen lediglich die GPU Unterstützung in Google Colab aktivieren.

**Ihre Aufgaben**

(1) Aktivieren Sie bitte die GPU Unterstützung in Google Colab. Wechseln Sie unter dem Reiter `Laufzeit` -> `Laufzeittyp ändern` von `None` auf `GPU` und bestätigen Sie diese Änderung.

(2) Führen Sie die folgende Codezelle aus, um die erforderlichen Bibliotheken zu importieren und die GPU Unterstützung zu prüfen. 

  * Hinweis: Wenn Sie dieses Notebook mit dem Direktlink von GitHub geöffnet haben, wird bei der erstmaligen Ausführung eine Warnung angezeigt. Diese müssen Sie durch den Klick auf `Trotzdem ausführen` bestätigen. 

In [ ]:
# Import everything needed for this exercise 
import tensorflow as tf
import keras
import numpy as np
import sys
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
from termcolor import colored
import logging

tf.get_logger().setLevel(logging.ERROR)
print(f"Keras Version: {keras.__version__}; Tensorflow version: {tf.__version__}; NumPy version: {np.__version__}; Python version: ", ".".join(str(x) for x in sys.version_info[:3]))

# Reset random number generators
seed = 1337
np.random.seed(seed)
tf.random.set_seed(seed)

# Check GPU support
from tensorflow.python.client import device_lib
physical_devices = device_lib.list_local_devices()
print("You are using", len(physical_devices), "local devises.", len([x for x in physical_devices if x.device_type == "GPU"]), "are GPUs")
for i, d in enumerate(physical_devices):
    print("  -> Device", i+1, "is a", d.device_type, "=>", d.physical_device_desc if len(d.physical_device_desc) > 0 else d.name)

if len([x for x in physical_devices if x.device_type == "GPU"]) == 0:
  raise Exception("Please enable GPU support before using this notebook. See here: [Runtime] -> [Change runtime type]")    

print("\n🎉🎉🎉 You are ready to go! 🎉🎉🎉")  

## 7.1 One Hot Kodierung

Wir haben in der Vorlesung die One Hot Kodierung (1-aus-n-Code) kennen gelernt. Diese ermöglicht es Zeichen eines Alphabets als Vektoren darzustellen. 

Für ein Alphabet mit `n` einzigartigen Zeichen wird ein Vektor der Länge `n` benötigt. Jedes einzigartige Zeichen wird hierbei als Einheitsvektor definiert. Eine Zeichensequenz mit `m` Zeichen wird als `m` Vektoren mit jeweils der Länge `n` dargestellt.  

**Hinweis** Die vorliegende Implementierung ist für den Umgang mit Zeichen außerhalb des Alphabets konzipiert. Deshalb wird zur Darstellung eines Alphabets mit `n` einzigartigen Zeichen ein Vektor der Länge `n+1` erstellt. Alle Zeichen außerhalb des Alphabets werden durch das `unknown_token` ersetzt. 

Für das Training unseres Netzes werden wir die One Hot Kodierung zur Umwandlung eines Textes in Trainingsdaten nutzen. (Aufgabe 7.2)

* Hinweis: Für weitere Informationen zur One Hot Kodierun schlagen Sie diese in der Vorlesung nach oder benutzen Sie das [Internet](https://www.semanticscholar.org/search?q=one%20hot%20coder&sort=relevance).

In [ ]:
class OneHot(object):
    def __init__(self, tokens, unknown_token = "[UNKNOWN]"):
        '''
        Creates a new instance of this class. 

        OneHot can be used to translate categorical data into vectors.

        Parameters
        tokens -- The tokens you want to be able to encode and decode
        unknown_token -- The token to be used when decoding and the net wants to use a not known char
        '''
        self.tokens = tokens
        self.unknown_token = unknown_token
        # Store a bidirectional dictionary containing the characters
        self.char_to_index = dict((token, i + 1) for i, token in enumerate(self.tokens))
        self.index_to_char = dict((i + 1, token) for i, token in enumerate(self.tokens))
        pass

    def encode(self, text_as_tokens, dtype=np.bool_):
        '''
        Encodes a tokenized text into a matrix (each token as vector).

        Parameters:
        text_as_tokens -- List of tokens
        dtype -- (optional) data type like int or bool for optimized performance

        Returns:
        np.array of OneHot encoded tokens (matrix)
        '''
        # Create the encoding matrix
        enc = np.empty((len(text_as_tokens), len(self.tokens) + 1), dtype=dtype)
        for i, token in enumerate(text_as_tokens):
            # Encode every char
            enc[i] = self.encode_token(token=token, dtype=dtype)
        return enc
    
    def encode_token(self, token, dtype=np.bool_):
        '''
        Encodes a single token into a vector.

        Parameters:
        token -- Single token
        dtype -- (optional) data type like int or bool for optimized performance

        Returns:
        np.array of OneHot encoded token (vector)
        '''
        l = len(self.tokens) + 1
        ret = np.zeros((1, l), dtype=dtype)
        if token not in self.char_to_index:
            ret[0, 0] = 1
        else:
            ret[0, self.char_to_index[token]] = 1
        return ret
    
    def decode(self, mat, unknown_token=None):         
        '''
        Decodes a matrix into an array of tokens.

        Parameters:
        mat -- matrix to be decoded. Has to be of shape (len_of_text, vocab_size)
        unknown_token -- Unknown token. If none uses the one from __init__

        Returns:
        array of tokens (chars)
        '''
        return [self.decode_token(mat[x]) for x in range(mat.shape[0])]
    
    def decode_token(self, vec, unknown_token=None):
        '''
        Decodes a vector into a token.

        Parameters:
        vec -- Vector that should be decoded. Has to be on vocab length
        unknown_token -- Unknown token. If none uses the one from __init__

        Returns:
        single token (char)
        '''
        if unknown_token is None:
            unknown_token = self.unknown_token
        if isinstance(vec, tf.Tensor):
            vec = vec.numpy()
        if isinstance(vec, np.ndarray):
            # Use argmax since this will be used in the model created later
            am = np.argmax(vec)
        else:
            am = vec
        if am == 0:
            return unknown_token
        return self.index_to_char[am]

**Ihre Aufgaben**

(1) Untersuchen Sie die gegebene Klasse zur One Hot Kodierung und die Funktion der einzelnen Methoden. 

(2) Erstellen Sie für das folgendes Alphabet `HWedlor` eine Instanz der Klasse `OneHot`. 

(3) Kodieren Sie nun die Zeichenfolge `Hello World!` mit der One Hot Kodierung. Wie wird das Zeichen `H` dargestellt? Geben Sie die Dimensionen des Vektors an.

* Hinweis: Sie können zur Umwandlung des Strings in eine Liste von Zeichen die Funktion `tokenize` verwenden. Dies ist aber nicht zwingend erforderlich, da Python den String automatisch als Liste interpretieren kann.

> <Antwort hier einfügen>

(4) Wandeln Sie die kodierte Zeichenfolge zurück in einen für Menschen lesbaren Text. Entspricht der zurückgewandelte Text dem ursprünglichen Input? Falls nicht, was könnte die Ursache dafür sein?

* Hinweis: Für eine schönere Ausgabe kann der enkodierte Text mit der Funktion `untokenize` in einen String gewandelt werden.

> <Antwort hier einfügen>

In [ ]:
def tokenize(text):
    '''
    Converts the given text (string) in a list of chars.
    '''
    return list(text)

def untokenize(tokens):
    '''
    Converts the given tokens (list of chars) in a string.
    '''
    return "".join(tokens)

tokens = 'HWedlor'

# code here




## 7.2 Datensatz vorbereiten

In dieser Übung verwenden wir einen Auszug aus dem frei zugänglichen Buch [Shakespeare](http://shakespeare.mit.edu/) als Datensatz. Ziel des Modells soll die Vorhersage weiterer Zeichen zu einer gegebenen Zeichensequenz sein.

Sie können aus einem Buch (sehr lange Zeichensequenz) Trainingsdaten erzeugen, indem Sie jedem Zeichen das jeweils nächste Zeichen als Label zuweisen. Die folgende Grafik veranschaulicht dieses Verfahren:
![Image](https://raw.githubusercontent.com/fhac-ewi/recurrent-neural-network/main/TrainData.png)

In dieser Aufgabe wird das Buch einlegesen, mithilfe der One Hot Kodierung (Aufgabe 7.1) in Vektoren umgewandelt, in Teilsequenzen unterteilt und als Trainingsdaten und Validierungsdaten aufgeteilt.

* Hinweis: Sie können theoretisch jeden beliebigen Text als Datensatz für das Training des Modells verwenden, wie z.B. die gesamten Harry Potter Bücher, Coronaschutzverordnungen oder auch Nachrichtenartikel.

**Ihre Aufgaben**

(1) Führen Sie die folgende Codezelle aus, um das Buch Shakespeare einzulesen. Untersuchen Sie den Datensatz, indem Sie einen kurzen Auszug des Textes ausgeben.


In [ ]:
# maximum text length 
# 1. Protection against "OutOfMemory" (poor RAM 😥)
# 2. Adjustment for making training faster (but you will gain less accuracy)
MAX_TEXT_LEN = 1_200_000 

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = open(path_to_file, 'rb').read().decode(encoding='utf-8') # .lower() für schnelleres Training anfügen
print(f'Length of text: {len(text)} characters')
if len(text) > MAX_TEXT_LEN:
    print("Text is too long. Cutting it to", MAX_TEXT_LEN, f"characters. That is {100 * MAX_TEXT_LEN / len(text):6.2f} %")
    text = text[:MAX_TEXT_LEN]

# code here   




(2) Wandeln Sie den `text` in Tokens um (`tokenized_text`), indem Sie die Funktion `tokenize` anwenden.

In [ ]:
# code here




(3) Ermitteln Sie aus `tokenized_text` das Alphabet (`tokens`) und geben dessen Länge aus. Das Alphabet soll alle einzigartigen Zeichen des Textes enthalten.

* Hinweis: Die Dokumentation von [Set](https://docs.python.org/3/tutorial/datastructures.html#sets) kann Ihnen bei dieser Aufgabe weiterhelfen.

In [ ]:
# code here




(4) Erstellen Sie für das Alphabet eine One Hot Kodierung und wenden diese auf den `tokenized_text` an. Nutzen Sie dazu Ihre Erkentnisse aus der Aufgabe 7.1.

In [ ]:
# code here





(5) Teilen Sie den kodierten Text (der ein großer Datensatz ist) in Sequenzen (also in mehrere kleinere Datensätze) auf. Wie würden Sie die Länge der einzelnen Sequenzen wählen? 
Verwenden Sie dazu die vorgegebene Funktion `sequenze_split`.

* Hinweis: Die Sequenzen sollten lang genug sein, dass das RNN Zusammenhänge in einem Satz (und ggf. darüber hinaus) erlernen kann. Die Sequenzlänge sollte jedoch kurz genug sein, damit eine ausreichende Anzahl an Datensätzen für das Training vorhanden ist.
<br>Die von uns verwendete Sequenzlänge haben wir mit [Rot13](https://rot13.de/) kodiert: `Rvauhaqreg`

In [ ]:
# helper function for 7.2.5
def sequenze_split(coded_text, sequence_len):
    '''
    Splits a given coded text (text converted with OneHot) into multiple sequences.

    Parameters:
    coded_text -- OneHot coded text (2D np.array) that will be splitted.
    sequence_len -- Length of each sequence

    Returns:
    3D np.array with coded text splitted into sequences.
    Shape will be (sequences, letters per sequence, letter as one hot coded vector)
    '''
    target_shape = (int(coded_text.shape[0] / (sequence_len + 1)) , (sequence_len + 1) , coded_text.shape[1])
    coded_text_seq = np.empty(target_shape, dtype=coded_text.dtype)
    for s in range(coded_text_seq.shape[0]):
        coded_text_seq[s] = coded_text[s * (sequence_len + 1):(s + 1) * (sequence_len + 1)]
  
    return coded_text_seq  

# code here




(6) Erstellen Sie nun aus den sequenzierten Daten das Feature `X` und das Label `y`. Für die Vorhersage von einzelnen Zeichen ist jeweils das nächste Zeichen das Label des vorherigen Zeichens. Das Label `y` wird deshalb aus den gleichen Werten des Features `X` gebildet, ist jedoch um +1 verschoben. 


* Beispiel: Die Sequenz `Hello World!` kann in **X** `Hello World` mit dem zugehörigen Label **y** `ello World!` aufgeteilt werden.
* Hinweis: Untersuchen Sie die Rückgabe der Funktion `sequenze_split`. In welcher der drei Komponenten müssen Sie die Verschiebung vornehmen?


In [ ]:
# code here





(7) Konvertieren Sie nun die Daten in ein Trainings- und Validierungsset. Nutzen Sie dafür die Funktion `train_test_split` (Eine Dokumentation finden Sie [hier](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)). 

In [ ]:
# code here




🎉🎉🎉 **Geschafft!** 🎉🎉🎉

Sie haben nun aus einem Buch Trainingsdaten für ein RNN zur Vorhersage von Zeichen erstellt!
Folgende Schritte haben Sie erfolgreich durchgeführt: 
- Text einlesen (und falls notwendig bereinigen)
- Text in Tokens umwandeln
- Alphabet festlegen
- One Hot Kodierung erstellen und anwenden
- In Sequenzen aufteilen
- `X` und `y` festlegen
- Aufteilung in Training und Validierungsset

... jetzt können Sie mit dem eigentlichen Modell fortfahren.

## 7.3 RNN Modell erstellen
In dieser Aufgabe wird ein many-to-many [SimpleRNN](https://keras.io/api/layers/recurrent_layers/simple_rnn/) (auch als Vanilla bezeichnet) zur Vorhersage des weiteren Verlaufs einer Zeichensequenz erstellt.

Für die Erstellung eines Vanilla RNN werden Sie ein Sequential Model von Keras mit einer SimpleRNN Layer und einer Dense Layer nutzen. 
![Image](https://raw.githubusercontent.com/fhac-ewi/recurrent-neural-network/main/SimpleRNN.png)

**Ihre Aufgaben**

(1) Bestimmen Sie nun die Parameter, die für das Modell benötigt werden:
- Shape der Eingabe/des Trainingdatensatzes. (Anzahl Zeichen, Länge eines vektorisierten Zeichens)
- Länge des Alphabets (inklusive des `unknown_tokens`). Also die Anzahl der Tokens, die das Modell vorhersagen können soll.
  


In [ ]:
# code here





(2) Erstellen Sie ein [Sequential](https://keras.io/api/models/sequential/) Modell und fügen folgende Schichten hinzu:
- [SimpleRNN](https://keras.io/api/layers/recurrent_layers/simple_rnn/) mit 128 RNN Units als Output Space (= Größe des Hidden State), der Shape der Eingabe und dem Parameter `return_sequences=true`. Wenn wir den Parameter `return_sequences=true` setzen, gibt unsere SimpleRNN-Schicht die gesamte Output-Sequenz zurück (Das Resultat entspricht einem `many-to-many RNN`).
- [Dense](https://keras.io/api/layers/core_layers/dense/) mit der Anzahl der Tokens und der Aktivierungsfunktion `softmax`.

Kompilieren Sie anschließend das Modell mit der Loss Function `categorical_crossentropy`, dem [RMSprop](https://keras.io/api/optimizers/rmsprop/) Optimizer mit einer `learning_rate` von 0.01 und fügen als Metrics die `accuracy` hinzu.

In [ ]:
# code here




## 7.4 RNN Modell trainieren
In dieser Aufgabe soll das Modell trainiert werden sowie der Loss und die Accuracy untersucht werden.

**Ihre Aufgaben**

(1) Wenden Sie die Methode [fit](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) auf Ihr Modell an. Geben Sie als Parameter neben den Trainingsdaten auch die Anzahl der Epochen, die Batchsize und die zuvor erstellten Validierungsdaten (Aufgabenteil 7.2.6) an. Speichern Sie das Ergebnis des Trainings für eine spätere Analyse in einer Variable!

Im Code sind bereits Startwerte für das Training vorgegeben. Sie können diese nach Belieben anpassen. 

**Eine Accuracy von 50% ist für diesen Schritt völlig ausreichend!**

* Hinweis: **Die Trainingsdauer sollte zwischen 1 und 3 Minuten liegen!** 
</br>Nach den ersten drei Epochen sollte die Genauigkeit bereits über 30% liegen. Falls nicht, prüfen Sie noch einmal die vorherigen Aufgabenteile oder bitten Sie das Team RNN um Hilfe!

* Tipp: Falls Sie das Gefühl haben, dass Ihr Modell noch einen Fehler enthalten könnte, fügen Sie in Aufgabe 7.2 im Codeblock Zeile 8 ein `.lower()` ein. Dadurch wird der Datensatz in Kleinbuchstaben umgewandelt und die Größe des Alphabets verringert. 

In [ ]:
EPOCHS = 20
BATCH_SIZE = 128

t1 = time.time()
# code here



t2 = time.time()

print("Training took: ", timedelta(seconds=t2-t1))

(2) Plotten Sie die Accuracy und den Loss des Trainingsets und des Validationsets über die Epochen hinweg. Bewerten Sie den Verlauf des Trainings. Wie könnte das Training verbessert werden?
* Hinweis: Die Accuracy und den Loss finden Sie in der [history](https://keras.io/api/models/model_training_apis/#fit-method) des Modells. Schauen Sie sich hierzu die Rückgabe der `fit-Methode` an.

In [ ]:
# code here




> <Antwort hier einfügen>


## 7.5 Vorhersagen treffen
In diesem Schritt wollen wir unser zuvor trainiertes Modell für die Vorhersage von Buchstaben/Texten nutzen. Basierend auf den resultierenden Sätzen können wir eine von der Accuracy unabhängige Bewertung der Modellgüte erstellen.

**Ihre Aufgaben**

(1) Vervollständigen Sie die Funktion `predict`. Kodieren Sie dafür den Text mit der One Hot Kodierung, führen die Vorhersage mit dem trainierten Modell durch und geben das letzte dekodierte Zeichen der Vorhersage zurück.  

* Hinweis: Beachten Sie, dass die Methode `model.predict` immer Batches erwartet und der Input in eine andere Shape gebracht werden muss.

Testen Sie anschließend die Funktion mit einzelnen Wörtern. Geben Sie dafür den ersten Teil des Wortes in die `predict` Funktion hinein und überprüfen die Rückgabe. Sie können auch kürzere Sätze hineingeben und die Ausgabe prüfen.

* Beispiel: Bei `Prince` wird der Input als `Princ` gewählt und das erwartete Ergebnis lautet `e`.

Wir haben Ihnen zusätzlich ein Widget mithilfe von [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html#Using-Interact) programmiert, damit Sie verschiedene Inputs live testen können.

In [ ]:
def predict(text, one_hot, trainied_model):
    '''
    Wandelt den Text mithilfe der OneHot Kodierung in eine Input um, 
    führt mithilfe des trainierten Modells eine prediction durch 
    und gibt das vorhergesagte Zeichen als Text aus.
    '''
    # coce here
    


    raise NotImplementedError

print("Prediction of 'fathe' is:", predict("fathe", one_hot, model))   
print("Prediction of 'Princ' is:", predict("Princ", one_hot, model))  

In [ ]:
import ipywidgets as widgets
from IPython.display import display
def verbose_prediction(text, one_hot, trainied_model):
    if len(text) == 0:
        text = " "
    prediction = predict(text=text, one_hot=one_hot, trainied_model=trainied_model)
    print(f"I think the next will be \"{prediction}\" after you said \"{text}\"")

w = widgets.interactive(verbose_prediction,
                        text=widgets.Text(value='ROME', placeholder='Type something', description='Your text:', disabled=False),
                        one_hot=widgets.fixed(one_hot),
                        trainied_model=widgets.fixed(model),
                       )

verbose_prediction("Edwar", one_hot=one_hot, trainied_model=model)

display(w)

(2) **Wettkampf** Erweitern Sie die Funktion `predict` aus dem vorherigen Aufgabenteil (7.5.1) zu einer Funktion, die 100 weitere Zeichen vorhersagt und somit womöglich einen ganzen Satz vorhersagen kann. 

Reichen Sie Ihr bestes Ergebnis im [Discord Chat von Team RNN](https://discord.com/channels/829659610045481010/829659610210107434/846648596681195550) ein. Wenn Sie in den nachfolgenden Aufgaben bessere Ergebnisse erzielen, können Sie diese natürlich auch einreichen!

In [ ]:
def predict_sentence(text, prediction_length, one_hot, trainied_model):
    '''
    Führt mehrere Predictions Iterativ durch.
    '''
    # code here
    

    
    return text

print(predict_sentence("ROMEO:\n", 100, one_hot, model))      

## 7.6 **Optional** RNN Varianten LSTM und GRU

Neben SimpleRNN wurden in der Vorlesung zwei weitere Varianten von RNNs vorgestellt, die mit dem Vanishing Gradient Problem deutlich besser als das SimpleRNN umgehen können.

In dieser Aufgabe sollen diese beiden Varianten trainiert und die Lernkurve sowie die getroffenen Vorhersagen mit dem SimpleRNN verglichen werden.

**Ihre Aufgaben**

(1) Trainieren Sie auf dem gleichen Datensatz zwei Modelle der Varianten LSTM und GRU. Verwenden Sie Ihr Modell aus Aufgabe 7.3 und ersetzen Sie die SimpleRNN Layer durch eine passende [Layer von Keras](https://keras.io/api/layers/recurrent_layers/).

**Speichern Sie die Modelle in neuen Variablen ab!**


In [ ]:
# code here
# model_lstm = ....



# model_gru = ....




In [ ]:
t1 = time.time()
# code here (train lstm)



t2 = time.time()
t_lstm = t2 - t1

t1 = time.time()
# code here (train gru)



t2 = time.time()
t_gru = t2 - t1

print("LSTM Training took:", timedelta(seconds=t_lstm))
print("GRU  Training took:", timedelta(seconds=t_gru))

(2) Plotten Sie die Accuracy und den Loss der drei Modelle in einem Diagramm. Verwenden Sie zusätzlich die Methode `predict_sentence` (7.5.2), um die Vorhersagen zu vergleichen.
Welche Erkenntnisse haben Sie aus dem Training gezogen? 

> <Antwort hier einfügen>


In [ ]:
# code here




## 7.7 **Optional** Schriftsteller

Mit den Erkenntnissen aus der Vorlesung, den vorherigen Übungsaufgaben und ein bisschen Python Erfahrung sind Sie nun in der Lage nicht nur einzele Buchstaben vorherzusagen, sondern könnten auch beliebig viele Zeichen generieren und dadurch ganze Bücher schreiben [lassen].

Sie können für diese Aufgabe den ursprünglichen Datensatz anpassen und beispielsweise die Coronaschutzverordnung, wissenschaftliche Artikel oder Meldungen aus Tageszeitungen verwenden.

Gelingt es Ihnen den 8ten Band von Harry Potter zu erstellen?

**Ihre Aufgaben**

(1) Was müssten Sie ändern, um mithilfe des bisherigen Modells ein ganzes Buch erstellen zu lassen? Glauben Sie, dass das Ergebnis lesbar wäre?

> <Antwort hier einfügen>


(2) Erstellen Sie ein neues Modell, welches zu einem gegebenen Text von wenigen Worten in der Lage ist, ein ganzes Buch vorherzusagen. Ihnen sind keine Grenzen gesetzt. Sie sind jetzt frei! 🧦 

In [ ]:
# code here


